# 0 Information

## 0.1 Prerequisites

### 0.1.1 Git
* Should be installed
* Should be available via path, so that it is possible to call `git clone ...` directly from the console

### 0.1.2 Python
It is recommended to use version 3.8+. The oldest version on which
this notebook and all codebase scripts have been tested is 3.7.

### 0.1.3 tar
This tool is required if you want to download the dataset and unzip it

## 0.2 Recommendations

### 0.2.1 Configuration
The current setup of this file allow you to completely run all the modules of the project without any problems.
If you wish to simply execute all the functions, and you don't require special configuration or output directories
then please stick to the configuration that is provided here by default.

### 0.2.2 Google colab
A lot of scripts from here are very slow and it is advisable to run them from Google Colab. Also the whole testing was done on colab so it is a guarantee that everything works on Colab as intended. In Colab you don't have to care about virtual environments and packages in python.

### 0.2.3 Virtual environment
If you still decide to run the notebook locally, it is recommended to run this notebook in a separate `conda` python virtual environment.
Even though this is not a hard requirement to run this notebook it is very recommended using a virtual environment.

## 0.3 App
This notebook does not contain the startup of the frontend for the animal detector app. For this, follow the instructions from the readme.md file of the github repo for this project.

# 1 Setup

## 1.1 Constants

In [ ]:
MODE_VERBOSE = 'verbose'
MODE_SILENT = 'silent'

DEFAULT_TRAINED_MODEL = 'my_model'

DEFAULT_CONFIG_FILE = 'config_default.json'

GITHUB_REPO_URL = 'https://github.com/Philideli/animal_detection.git'

## 1.2 Adjustable config constants
Change these if you want to experiment with the functionality of this notebook

### `SCRIPTS_EXEC_MODE`
It is possible to execute all scripts in `verbose` mode to display
all logs that are displayed by the scripts themselves
and also by all the subprocesses that they start. However, if you use `silent` mode
then no logs will be displayed. Only important messages like errors and
important execution checkpoints will be logged in silent mode.

In [ ]:
SCRIPTS_EXEC_MODE = MODE_VERBOSE # MODE_SILENT

### `EVAL_SPLIT_RATIO`
Specify a float value from `0.0` to `1.0` which defines how big the part of images should
be used for evaluation. Value of `0.2` means that 20% of all the images in the provided dataset
should be used for evaluation. The rest is used for training

In [ ]:
EVAL_SPLIT_RATIO = 0.2

### `NUM_TRAIN_STEPS`
The number of training steps during the model training process. See TFOD API documentation for details.

In [ ]:
NUM_TRAIN_STEPS = 20000

### `CHECKPOINT_EVERY_N`
How often should Tensorflow make checkpoints of training status. See TFOD API documentation for details.

In [ ]:
CHECKPOINT_EVERY_N = 1000

### `EXISTING_MODEL`

If you do not need to train the model, please specify the name of the trained model here and put its weights in the model

In [ ]:
EXISING_MODEL = None # set to DEFAULT_TRAINED_MODEL if my_model is present and no training is needed

### `DOWNLOAD_DATASET`
Set to `True` if the dataset should be downloaded from the servers (https://thor.robots.ox.ac.uk/~vgg/data/pets/images.tar.gz and https://thor.robots.ox.ac.uk/~vgg/data/pets/annotations.tar.gz)

In [ ]:
DOWNLOAD_DATASET = True

# 2 Workspace preparation
All of these cells should be run only for one time when we "start from scratch" with this notebook
Install the project package to the python (virtual) environment

In [ ]:
!git clone {GITHUB_REPO_URL} ./animal_detection

import shutil
shutil.copy('./animal_detection/setup.py', './setup.py')

In [ ]:
import pip
pip.main(['install', '-e', '.'])

In [ ]:
%cd ./animal_detection/

In [ ]:
! python -m pip install django django-cors-headers django-rest-framework django-composite-field mysqlclient

## 2.1 Download archive with data

In [ ]:
if DOWNLOAD_DATASET:
    import urllib, os
    urllib.request.urlretrieve("https://thor.robots.ox.ac.uk/~vgg/data/pets/annotations.tar.gz", "annotations.tar.gz")
    print('downloaded annotations')
    urllib.request.urlretrieve("https://thor.robots.ox.ac.uk/~vgg/data/pets/images.tar.gz", "images.tar.gz")
    print('downloaded images')
    
    os.makedirs('./data/datasets/raw/my_dataset/annotations', exist_ok=True)
    os.makedirs('./data/datasets/raw/my_dataset/images', exist_ok=True)
    !tar -zxf ./annotations.tar.gz --directory ./data/datasets/raw/my_dataset/annotations
    !tar -zxf ./images.tar.gz --directory ./data/datasets/raw/my_dataset/images
    

# 3 TFOD API and pretrained model
Here we install Tensorflow Object Detection API and
load the pretrained model that is specified in the config.
Generally, the TFOD API should only be installed once.
However, several pretrained models can be loaded (when the config is changed)
and in this case it is required to run these cells once again
for the new pretrained model to be used during training.

In [ ]:
import os
python_file = os.path.join('setup', 'workspace.py')
! python {python_file} -m {SCRIPTS_EXEC_MODE} -v "both"

In [ ]:
# the code above installs some packages with incorrect versions and
# we need to reinstall these packages with correct versions
# for correct functionality of other parts of this notebook
! python -m pip install matplotlib scikit-learn pillow pytz gin-config tensorflow[and-cuda]==2.13.0 opencv-python

# it sometimes also helps to rerun the cell above after running this one

## 4 Dataset preparation
Here we create the processed dataset with splitting into train and test sets.
Please put your raw data in ./data/datasets/raw/my_dataset if not done in the previous step

In [ ]:
import os
python_file = os.path.join('data', 'preparation.py')
!python {python_file} -m {SCRIPTS_EXEC_MODE} -sr {EVAL_SPLIT_RATIO}

## 5 Train the model
Train the model on the created TF records

In [ ]:
import os
python_file = os.path.join('models', 'train.py')
!python {python_file} -m {SCRIPTS_EXEC_MODE} --num_train_steps {NUM_TRAIN_STEPS} --checkpoint_every_n {CHECKPOINT_EVERY_N}

## 6 Evaluate the model

In [ ]:
import os
python_file = os.path.join('models', 'eval.py')
!python {python_file} -m {SCRIPTS_EXEC_MODE}

## 7 Image Object Detection

In [ ]:
IMAGE_PATH = r'put the path for the image that you want to execute detection on'

In [ ]:
import os
python_file = os.path.join('detect', 'detect_objects.py')
!python {python_file} --image-path "{IMAGE_PATH}"